# Machine learning for loan default prediction

## EDA

In [1]:
import pandas as pd
import os
import sys
import pprint
import matplotlib.pyplot as plt
import scipy.stats as scistat
import math
import seaborn as sns

path_to_modules = r"C:\Users\Andres Ruiz Rada\Documents\GitHub\poc-069-lumon-riesgo-crediticio-fase1\datalab"
sys.path.append(path_to_modules)
import basicdescriptives_mod as bd

### Carga de dataset y exploracion inicial de columnas

In [2]:
df = pd.read_excel(r"C:\Users\Andres Ruiz Rada\Documents\GitHub\poc-069-lumon-riesgo-crediticio-fase1\data\raw\db_v0.xlsx")
df.head()

,ID_USUARIO,ID CREDITO,PRODUCTO,MONEDA,CUOTAS,PERIODICIDAD CUOTAS,PLAZO,TASA CORRIENTE,TASA SEGURO,TASA AVAL,...,GASTOS MENSUALES,INGRESOS ADICIONALES,TIPO DE CONTRATO,PERIODO DE PAGO,ESTADO CIVIL,NIVEL EDUCATIVO,PERSONAS A CARGO,NUMERO DE HIJOS,TIPO DE VEHICULO,TIEMPO TRABAJO
0,1,136,CREDI-ONE,COP,1,9,9,0.01877,0.0024,0,...,NaN,NaN,Indefinido,Mensual,NaN,NaN,0.0,0.0,Ninguno,24 meses
1,2,133,CREDI-ONE,COP,1,14,14,0.01877,0.0024,0,...,NaN,NaN,Indefinido,Mensual,NaN,NaN,0.0,0.0,Ninguno,12 meses
2,3,45,CREDI-ONE,COP,1,30,30,0.01877,0.0024,0,...,NaN,NaN,Termino fijo,Mensual,NaN,NaN,0.0,0.0,Carro,1 año
3,4,78,CREDI-ONE,COP,1,29,29,0.01877,0.0024,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,Ninguno,NaN
4,5,114,CREDI-ONE,COP,1,28,28,0.01877,0.0024,0,...,NaN,NaN,Termino fijo,Mensual,NaN,NaN,1.0,1.0,Ninguno,9 horas


### Resumen del dataset crudo

El dataset contiene 119.243 registros y 60 caracteristicas.

El dataset contiene datos de clientes que han solicitado creditos, los datos incluyen información financiera y socio-demográfica. El objetivo es predecir si un cliente es un buen candidato para un préstamo o no, utilizando las características disponibles en los datos. Esto puede ser útil para los prestamistas para evaluar el riesgo de otorgar un préstamo a un cliente en particular y tomar decisiones informadas sobre la aprobación de préstamos.

Algunas de las características incluidas en el dataset son:

- ID Credito
- ID Usuario anonimizado
- Edad
- Ingresos anuales
- Ocupacion
- Estado civil

El total de los registros corresponden a 23948 clientes y se tienen 3249 ID de credito duplicados

In [10]:
info_df = bd.info(df)
print(info_df)

Number of Columns: 60
Number of total rows: 119243
Number of Unique Rows: 119243
             Column Name Data Type  Non null values  Unique Values
0             ID_USUARIO     int64           119243          23948
1             ID CREDITO     int64           119243         115994
2               PRODUCTO    object           119243              1
3                 MONEDA    object           119243              1
4                 CUOTAS     int64           119243              1
5    PERIODICIDAD CUOTAS     int64           119243             30
6                  PLAZO     int64           119243             30
7         TASA CORRIENTE   float64           119243              2
8            TASA SEGURO   float64           119243              1
9              TASA AVAL     int64           119243              1
10              IVA AVAL     int64           119243              1
11             DESC AVAL     int64           119243              1
12   DESC AVAL AL DESEMB     int64           119

### Eliminacion de columnas con  valor unico
Para optimizar la exploracion y procesamiento se eliminan las columnas sin varianza, dando como resultado un dataset con 44 caracteristicas

In [11]:
# Filtrado columnas sin varianza, 'Unique Values' == 1
df_filtered_uniqueV = info_df[info_df['Unique Values'] == 1]

# Extract 'Column Name' and convert to list
columns_with_unique_one = df_filtered_uniqueV['Column Name'].tolist()
df_unique = df.drop(columns = columns_with_unique_one)

bd.info(df_unique)

Number of Columns: 44
Number of total rows: 119243
Number of Unique Rows: 119243


,Column Name,Data Type,Non null values,Unique Values
0,ID_USUARIO,int64,119243,23948
1,ID CREDITO,int64,119243,115994
2,PERIODICIDAD CUOTAS,int64,119243,30
3,PLAZO,int64,119243,30
4,TASA CORRIENTE,float64,119243,2
5,CAPITAL,int64,119243,93
6,INT CORRIENTE,float64,119243,488
7,SEGURO,int64,119243,93
8,FECHA DESEMBOLSO,object,119243,1813
9,VALOR DESEMBOLSADO,int64,119243,93


In [12]:
filtered_ids =df_unique.groupby('ID_USUARIO')['ID CREDITO'].count
filtered_ids = filtered_ids[filtered_ids['ID_credito_list'].apply(len) > 1]

f = pd.DataFrame(filtered_ids)
f.head() 

TypeError: 'method' object is not subscriptable

In [8]:
duplicates = info_df[df.duplicated('ID CREDITO', keep=False)]
duplicates = duplicates.sort_values(by=['ID CREDITO', 'FECHA DESEMBOLSO'])
duplicates = duplicates.reset_index(drop=True)
duplicates.to_excel(r'C:\Users\Andres Ruiz Rada\OneDrive - Universidad EAFIT\Documentos compartidos - Maestría_2024\General\PI1\Python_project\views\duplicates.xlsx', index=False)

NameError: name 'info_df' is not defined

In [ ]:
duplicated_mask= df.duplicated('ID CREDITO')
df[df[duplicated_mask]==True]
print(df)

In [5]:
# get the distribution of a variable
dist = ol.getdistprops(df.CAPITAL)
pprint.pprint(dist)

{'count': 119243,
 'kurtosis': -0.992755095430844,
 'mean': 353180.4308345144,
 'median': 350000.0,
 'normpvalue': '0.0: Reject Normal',
 'normstat': 0.8957375288009644,
 'skew': -0.0911071620153874,
 'std': 120057.90275267424}


C:\Users\Andres Ruiz Rada\anaconda3\Lib\site-packages\scipy\stats\_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [3]:
grouped_df = df.groupby('ID_USUARIO')['ID CREDITO'].apply(list).reset_index(name='List of ID_CREDITO')

# Display the resulting DataFrame
grouped_df.head(20)
#print(grouped_df)

,ID_USUARIO,List of ID_CREDITO
0,1,[136]
1,2,[133]
2,3,[45]
3,4,[78]
4,5,[114]
5,6,[89]
6,7,[91]
7,8,[92]
8,9,[216]
9,10,[167]


In [4]:
credits_per_user = df.groupby('ID_USUARIO')['ID CREDITO'].apply(list).reset_index(name='ID CREDITO')
# Filter users with more than one credit
users_with_multiple_credits = credits_per_user[credits_per_user['ID CREDITO'].apply(len) > 1]

In [8]:
print(users_with_multiple_credits)

       ID_USUARIO                    ID CREDITO
100           101                     [326, 83]
110           111                    [439, 201]
112           113                    [451, 418]
119           120                    [563, 386]
122           123                    [612, 397]
...           ...                           ...
23897       23898              [412560, 419097]
23920       23921                [73402, 77786]
23922       23923  [75727, 81364, 89934, 96723]
23923       23924                [75837, 87344]
23932       23933                [81354, 91274]

[13861 rows x 2 columns]
